Primer paso es leer BD del servidor donde se tenga la tabla de los pesometros antiguos con problemas.
En este caso es el servidor de google,
Primero vamos a ver si tenemos conexion al servidor , mostrando las tablas que tiene la bd

In [2]:
print("holaaa")

holaaa


In [3]:
import mysql.connector
import time
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter


db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("conexion a gc")
consulta="SHOW TABLES"
cur.execute(consulta)
for row in cur.fetchall():
    print (str(row[0]))
print (" termino la consulta")



conexion a gc
camiones_reg
cargador_reg
pesometroreparado2
pesometroreparado3
pesometros
 termino la consulta


Los datos en bruto estan en la tabla pesometro columna acc1 pesometro integral y acc2 pesometro grava 3/4.
row[1]=fecha, row[5]=integral, row[6]=grava3/4.>COn los siguiente vamos a describir la tabla

In [4]:
consulta="DESCRIBE pesometros"
cur.execute(consulta)
for row in cur.fetchall():
    print (str(row[0]))
print ("se termino la descripcion de pesometros ")


id
fecha
tph1
tph2
tph3
acc1
acc2
acc3
diff1
diff2
diff3
se termino la descripcion de pesometros 


vamos a leer el primer y ultimo registro de la tabla pesometro que queremo insertar en la tabla reparada.
es impportante esta consulta, ya que de aca se obtiene el primer registro acumulado para reststar al insertar 
en la nuev tabla,


In [5]:
db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("conexion a gc")

integralx=[]
gravax=[]
fechaIni="'2019-08-11 02:15:38'"
fechaFin="'2019-08-16 22:47:00'" 

#consula
tabla="pesometros"
consulta="SELECT * FROM "+tabla+" WHERE fecha >= "+fechaIni+ "ORDER BY id ASC LIMIT 1"
print(consulta)
cur.execute(consulta)
print ("El primer registro de la consulta ")
for row in cur.fetchall():
     print (str(row[0]) + " " +str(row[1])+" "+str(row[2]) + " " +str(row[3]))
     integralx.append(row[5])
     gravax.append(row[6])
print (" ")
print ("El ultimo registro de la consulta aaaaa")
tabla="pesometros"
consulta="SELECT * FROM "+tabla+" WHERE fecha <= "+fechaFin+ "ORDER BY id DESC LIMIT 1"
cur.execute(consulta)
for row in cur.fetchall():
     print (str(row[0]) + " " +str(row[1])+" "+str(row[2]) + " " +str(row[3]))
     integralx.append(row[5])
     gravax.append(row[6])
print (" ")
print(integralx)
print(gravax)
restaint=integralx[0]
restagrav=gravax[0]



conexion a gc
SELECT * FROM pesometros WHERE fecha >= '2019-08-11 02:15:38'ORDER BY id ASC LIMIT 1
El primer registro de la consulta 
54218882 2019-08-11 02:15:41 260 100
 
El ultimo registro de la consulta aaaaa
54314731 2019-08-16 22:45:07 0 0
 
[256732994400, 265377798100]
[84825164500, 87408469800]


Ahora vamos a reparar los datos


In [57]:
db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("conexion a gc")
#consula
tabla="pesometros"

integral=[]
grava=[]
restaint=integralx[0]
restagrav=gravax[0]

anteriorint=0  # registro anterior de la base antigua
anteriorgrav=0
contador=0
print(integralx)
print(gravax)



conexion a gc
[254724709600, 265377798100]
[84099887700, 87408469800]


In [ ]:
#de atras deben venir los arreglos 
#restaint=integralx[0]
#restagrav=gravax[0]
#que son los valores acumulados, los cuales vamos a venecitar para restar
integral=[]
grava=[]
restaint=integralx[0]
restagrav=gravax[0]
db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("vamos a comenzar la revision e insercion")
consulta="select * FROM "+tabla+" WHERE fecha between "+fechaIni+" and "+fechaFin+" "  
print(consulta)
cur.execute(consulta)

anteriorint=0  # registro anterior de la base antigua
anteriorgrav=0
contador=0
print(integralx)
print(gravax)


for row in cur.fetchall():
     #print (str(row[0]) + " " +str(row[1])+" "+str(row[2]) + " " +str(row[3]))
    if contador==0:
         anteriorint=row[5]
         anteriorgrav=row[6]
         anteriorintsql=0
         anteriorgravasql=0
    if row[5]-anteriorint>3000:

        print("entro aca")
        restaint=restaint+row[5]-anteriorint
        print ("cambio de Integral")
    if row[6]-anteriorgrav>3000:
        restagrav=restagrav+row[6]-anteriorgrav
              
   
    print(str(contador)+"  "+str(row[1])+"   "+str(row[5])+"   "+str(row[5]-restaint)+"---------"+str(row[6])+"   "+str(row[6]-restagrav))
    integral.append(row[5]-restaint)
    grava.append(row[6]-restagrav)
    #aca debo hacer el insert a la nueva tabla
    consulta="INSERT INTO `pesometroreparado3` (`hora`, `acc1`, `acc2`, `acc3`)VALUES ('"+str(row[1])+"', '"+str(row[5]-restaint-anteriorintsql)+"', '"+str(row[6]-restagrav-anteriorgravasql)+"', NULL);"
    #print(consulta)
    #print(anteriorint)
    cur.execute(consulta)
    db.commit()
    anteriorint=row[5]
    anteriorgrav=row[6]
    anteriorintsql=row[5]-restaint
    anteriorgravasql=row[6]-restagrav
    contador=contador+1

print("la produccion de integral es : ")
print(str(integral[len(integral)-1]))
print("la produccion de integral es en toneladas: ")
print(str(integral[len(integral)-1]*100/1000))

print("la produccion de grava es : ")
print(str(grava[len(grava)-1]))

print("la produccion de grava en toneladas es : ")
print(str(grava[len(grava)-1]*100/1000))

vamos a comenzar la revision e insercion
select * FROM pesometros WHERE fecha between '2019-08-11 02:15:38' and '2019-08-16 22:47:00' 
[256732994400, 265377798100]
[84825164500, 87408469800]
0  2019-08-11 02:15:41   256732994400   0---------84825164500   0
1  2019-08-11 02:15:44   256732994600   200---------84825164600   100
2  2019-08-11 02:15:47   256732994800   400---------84825164700   200
3  2019-08-11 02:15:50   256732995000   600---------84825164800   300
4  2019-08-11 02:15:53   256732995300   900---------84825164900   400
5  2019-08-11 02:15:56   256732995500   1100---------84825165000   500
6  2019-08-11 02:15:59   256732995800   1400---------84825165100   600
7  2019-08-11 02:16:02   256732996000   1600---------84825165100   600
8  2019-08-11 02:16:05   256732996300   1900---------84825165200   700
9  2019-08-11 02:16:08   256732996500   2100---------84825165300   800
10  2019-08-11 02:16:11   256732996700   2300---------84825165400   900
11  2019-08-11 02:16:14   2567329969

109  2019-08-11 02:21:22   256742191700   27000---------84828868400   13700
110  2019-08-11 02:21:25   256742191900   27200---------84828868500   13800
111  2019-08-11 02:21:28   256742192100   27400---------84828868600   13900
112  2019-08-11 02:21:31   256742192400   27700---------84828868700   14000
113  2019-08-11 02:21:34   256742192600   27900---------84828868800   14100
114  2019-08-11 02:21:37   256742192900   28200---------84828868900   14200
115  2019-08-11 02:21:40   256742193100   28400---------84828869000   14300
116  2019-08-11 02:21:44   256742193400   28700---------84828869100   14400
117  2019-08-11 02:21:47   256742193600   28900---------84828869200   14500
118  2019-08-11 02:21:50   256742193800   29100---------84828869300   14600
119  2019-08-11 02:21:53   256742194100   29400---------84828869400   14700
120  2019-08-11 02:21:56   256742194400   29700---------84828869500   14800
121  2019-08-11 02:21:59   256742194600   29900---------84828869600   14900
122  2019-08

217  2019-08-11 02:26:59   256761415500   50300---------84832588200   26300
218  2019-08-11 02:27:02   256761415700   50500---------84832588300   26400
219  2019-08-11 02:27:05   256761415900   50700---------84832588500   26600
220  2019-08-11 02:27:08   256761416100   50900---------84832588600   26700
221  2019-08-11 02:27:11   256761416300   51100---------84832588700   26800
222  2019-08-11 02:27:14   256761416600   51400---------84832588900   27000
223  2019-08-11 02:27:17   256761416800   51600---------84832589000   27100
224  2019-08-11 02:27:20   256761417000   51800---------84832589200   27300
225  2019-08-11 02:27:23   256761417300   52100---------84832589300   27400
226  2019-08-11 02:27:26   256761417500   52300---------84832589400   27500
227  2019-08-11 02:27:30   256761417800   52600---------84832589600   27700
228  2019-08-11 02:27:33   256761418000   52800---------84832589700   27800
229  2019-08-11 02:27:36   256761418200   53000---------84832589800   27900
230  2019-08

326  2019-08-11 02:32:35   256761440300   75100---------84840020100   38800
327  2019-08-11 02:32:38   256761440500   75300---------84840020200   38900
328  2019-08-11 02:32:41   256761440800   75600---------84840020300   39000
329  2019-08-11 02:32:44   256761441000   75800---------84840020400   39100
330  2019-08-11 02:32:47   256761441300   76100---------84840020500   39200
331  2019-08-11 02:32:51   256761441500   76300---------84840020600   39300
332  2019-08-11 02:32:54   256761441700   76500---------84840020700   39400
333  2019-08-11 02:32:57   256761441900   76700---------84840020800   39500
334  2019-08-11 02:33:00   256761442200   77000---------84840020900   39600
335  2019-08-11 02:33:03   256761442400   77200---------84840021000   39700
336  2019-08-11 02:33:06   256761442700   77500---------84840021100   39800
337  2019-08-11 02:33:09   256761442900   77700---------84840021200   39900
338  2019-08-11 02:33:12   256761443100   77900---------84840021400   40100
339  2019-08

434  2019-08-11 02:38:10   256770460900   95600---------84840032900   51600
435  2019-08-11 02:38:13   256770461200   95900---------84840033000   51700
436  2019-08-11 02:38:16   256770461400   96100---------84840033200   51900
437  2019-08-11 02:38:19   256770461600   96300---------84840033300   52000
438  2019-08-11 02:38:22   256770461900   96600---------84840033500   52200
439  2019-08-11 02:38:25   256770462100   96800---------84840033600   52300
440  2019-08-11 02:38:28   256770462400   97100---------84840033700   52400
441  2019-08-11 02:38:31   256770462700   97400---------84840033900   52600
442  2019-08-11 02:38:35   256770463000   97700---------84840034000   52700
443  2019-08-11 02:38:38   256770463300   98000---------84840034200   52900
444  2019-08-11 02:38:41   256770463500   98200---------84840034300   53000
445  2019-08-11 02:38:44   256770463800   98500---------84840034400   53100
446  2019-08-11 02:38:47   256770464100   98800---------84840034600   53300
447  2019-08